In [ ]:
# 주성분 분석 : 데이터 그래프에서 분산이 큰 방향으로 직선을 그었을 때 그 직선을 주성분이라고 한다.
# 주성분에 특정 좌표를 직각으로 선을 이으면 주성분 선 위에 한 점으로 투영되고 이걸로 2차원을 1차원으로 변경해서 데이터를 줄일 수 있다.
# https://ddongwon.tistory.com/114에 설명 잘 되어있다.
!wget https://bit.ly/fruits_300_data -O fruits_300.npy
import numpy as np
fruits = np.load('fruits_300.npy')
fruits_2d = fruits.reshape(-1, 100*100)

from sklearn.decomposition import PCA
pca = PCA(n_components=50) # 주성분 개수를 50개로 둔다.
pca.fit(fruits_2d)
print(pca.components_.shape) # (50,10000)

# 6-2에서 만들었던 함수
import matplotlib.pyplot as plt
def draw_fruits(arr, ratio=1): # 3차원 배열을 입력받으면 그려주는 함수, 배율은 그래프 크기
  n = len(arr) # n=샘플개수
  rows = int(np.ceil(n/10)) # 행의 개수
  cols = n if rows<2 else 10 # 열의 개수(행이 1개면 열의 개수는 샘플개수고 아니면 10개다.)
  fig, axs = plt.subplots(rows,cols,figsize=(cols*ratio,rows*ratio),squeeze=False)
  for i in range(rows):
    for j in range(cols):
      if i*10+j<n:
        axs[i,j].imshow(arr[i*10+j],cmap='gray_r')
      axs[i,j].axis('off')
  plt.show()


# 주성분을 이용해 transform() <-> inverse_transform()으로 차원을 줄였다 펴보기
draw_fruits(pca.components_.reshape(-1, 100, 100)) # (50,100,100)
print(fruits_2d.shape) # (300,10000)
fruits_pca = pca.transform(fruits_2d) # transform()으로 주성분 적용
print(fruits_pca.shape) # (300,50)
fruits_inverse = pca.inverse_transform(fruits_pca) # inverse_transform()으로 주성분 풀기
print(fruits_inverse.shape) # (300,10000)

fruits_reconstruct = fruits_inverse.reshape(-1, 100, 100)
draw_fruits(fruits_reconstruct) # 원본에 비해 살짝 흐려지긴 했으나 별 차이 없음

# 설명된 분산 : 주성분이 원본 데이터의 분산을 얼마나 잘 나타냈는지 시각화해보기
print(np.sum(pca.explained_variance_ratio_)) # 0.9215...
plt.plot(pca.explained_variance_ratio_)
plt.show()

# 다른 알고리즘과 같이 사용하기
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

target = np.array([0]*100 + [1]*100 + [2]*100)
from sklearn.model_selection import cross_validate
scores = cross_validate(lr, fruits_2d, target) # 교차 검증
print(np.mean(scores['test_score'])) # 0.99
print(np.mean(scores['fit_time'])) # 0.94
# 특성이 10000개라 300개의 샘플에서는 과대적합된 모델을 만들기 쉽다.

scores = cross_validate(lr,fruits_pca,target) # 교차 검증(차원 축소한 것)
print(np.mean(scores['test_score'])) # 1.0 정확도
print(np.mean(scores['fit_time'])) # 0.03 훈련시간
# 50개 특성만 사용했는데 정확도가 100%이고 훈련시간이 0.03초이다.
# 차원을 축소하면 저장공간 뿐만 아니라 훈련 속도도 높일 수 있다.

pca = PCA(n_components=0.5) # 설명된 분산의 50%에 달하는 주성분의 개수는 2개다.
pca.fit(fruits_2d)
print(pca.n_components_) # 2
fruits_pca = pca.transform(fruits_2d)
print(fruits_pca.shape) # (300, 2)

scores = cross_validate(lr,fruits_pca,target) # 교차 검증(차원 축소한 것, 주성분 2개)
print(np.mean(scores['test_score'])) # 0.99
print(np.mean(scores['fit_time'])) # 0.04

# 차원 축소된 데이터를 이용해 k-평균 알고리즘으로 클러스터를 찾아보자.
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3,random_state=42)
km.fit(fruits_pca) # 차원축소된 데이터
print(np.unique(km.labels_, return_counts=True)) # fruits_pca로 찾은 클러스터는 각각 110,99,91개이다.

for label in range(0,3): # 파인애플과 사과가 약간 혼동 있음.
  draw_fruits(fruits[km.labels_ == label])
  print("\n")

# 훈련 데이터의 차원을 줄이면 시각화하기 편하다. fruits_pca는 2개의 특성이 있어서
# 2차원으로 표현할 수 있다.
for label in range(0,3):
  data = fruits_pca[km.labels_ == label] # fruits_pca가 (300,2)이므로 data[:,0]은 모든 행 + 0열이다.
  plt.scatter(data[:,0], data[:,1])
plt.legend(['apple', 'banana', 'pineapple'])
plt.show()

Output hidden; open in https://colab.research.google.com to view.